### Segment Clustering neighborhoods of toronto

#### import packages

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import geocoder

import urllib

#### Define URL from where to get the data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#with open(url) as html_file:
#    soup = BeautifulSoup(html_file,"lxml")
    
#print(soup)
headers = {'User-Agent': 'Mozilla/5.0'}



#### Fetch the data from url

In [3]:
response = urllib.request.urlopen(url)
rawhtml = response.read()

#### add fetched data to soup object

In [4]:
soup = BeautifulSoup(rawhtml,"html.parser")
    
#print(soup.select('table'))

#### Get the table and clean it

In [5]:
pclist = []
prevPC = "" # stores the previous postal code to see if we need to append neighbourhoods or create a new row
currNeighbourhoods = ""
prevBorough = ""
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if ((b == "Not assigned" and n=="Not assigned") or b == "blank"):
           #skip row
            a="skipping"
        else:
            if (n=="Not assigned"):
                n = b; # set the neighbourhood equal to the borough
            
            if (pc == prevPC):
                # append neighbourhoods into comma delimited string
                currNeighbourhoods = currNeighbourhoods + ", " + n
                 
            else:
                # new postal code, so write out previous row
                if (prevPC != ""):
                    pclist.append([prevPC,prevBorough,currNeighbourhoods])
                prevPC = pc # set a new PrevPC value for next loop
                prevBorough = b
                currNeighbourhoods = n # start a new neighbourhoods string
# write out last row
pclist.append([prevPC,prevBorough,currNeighbourhoods])               
            
toronto_df = pd.DataFrame(pclist, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### get shape of table

In [6]:
toronto_df.shape

(103, 3)

#### Get latitude and longitude of each postal code

In [ ]:
'''
# !pip install geocoder
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

lat_lng_coords = None
# create lists to store our new lats and longs
lats = []
longs=[]
#loop through our dataframe and look up the lat/long of each postal code
for index, row in toronto_df.iterrows():
    #print (row["PostalCode"])
    postal_code=row["PostalCode"]
    geolocator = Nominatim(user_agent="foursquare_agent")
    # loop until you get the coordinates
    location = None
    while(location is None):
        
        location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code), timeout=100)
    latitude = location.latitude
    longitude = location.longitude
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        #lat_lng_coords = g.latlng

    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    lats.append(latitude)
    longs.append(longitude)
    #print ('postal code {} had latitude {} and longitude {}'.format(postal_code, latitude, longitude))

toronto_df['Latitude'] = lats
toronto_df['Longitude'] = longs
toronto_df
'''

In [8]:
## After trying a lot with above commented code ,
## I got the code from the discussion forum
# "https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/3/threads/BNqlp1qxEemUkAqlaf0TXA/replies/2pD_ZVzMEemXoQpaMIA2fg/comments/pi-GDnCzEem6JBJNWiPf3A


Lat_list=[]
Lng_list=[]
for i in range(toronto_df.shape[0]):
    address='{}, Toronto, Ontario'.format(toronto_df.at[i,'PostalCode'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [9]:
toronto_df['Latitude'] = Lat_list
toronto_df['Longitude'] = Lng_list
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195
6,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749055,-79.362227
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180


In [10]:
toronto_df.shape

(103, 5)

In [11]:
toronto_df.to_csv("toronto_df.csv", index = False)